# §16.1 系列データ

系列データはシーケンス (Sequence) と呼ばれる。
シーケンスはこれまでのデータ（例えば画像データ）と異なり、特定の順序で並んでいて、その順序も特徴量の一つであるということである。
そこで再帰ニューラルネットワーク（RNN)は、シーケンスをうまく扱うためにモデル化されている。

## シーケンスモデルのカテゴリ

- many to one
    - 入力データはシーケンスで、出力はスカラーであるようなもの。テキストの感情分析などで使用される。
- one to many
    - 入力データは固定長で、出力がシーケンスであるもの。画像キャプショニングがその一例。
- many to many
    - 入出力どちらもシーケンスであるもの。機械翻訳がその一例。

# §16.2 シーケンスモデルの構築

## BOTTを使った学習

$$
L = \sum_{t=1}^T L^{t}
$$

時間 $t$ の損失は、それ以前の全ての時間の隠れユニットに依存するので、勾配は次のように計算できる。


## SimpleRNN （単層RNN）の実装

- SimpleRNN
    - units : 
    - return_sequences : 各時系列ごとの出力を返すか、最後の時刻の出力だけを返すか。ここでは True にしているので、それまでの時系列全ての出力を返す。
    
- 入力の形状は `(None, None, 5)`
    - １つ目：バッチ次元（可変の場合はNone)
    - ２つ目：シーケンス（シーケンスが可変長の場合はNone)
    - 3つ目：特徴量

In [22]:
import tensorflow as tf

tf.random.set_seed(1)

rnn_layer = tf.keras.layers.SimpleRNN(units=2, use_bias=True, return_sequences=True)
rnn_layer.build(input_shape=(None, None, 5))

# 各層の重みを初期化する
w_xh, w_oo, b_h = rnn_layer.weights

print(w_xh.shape)
print(w_oo.shape)
print(b_h.shape)

(5, 2)
(2, 2)
(2,)


入力情報を作成する。ここでは $3\times 5$次元のデータを作成した。行方向に時間軸、列方向に特徴量の個数を取っている。なのでこの練習では、$t=0,1,2$ の時間軸に対してRNNを組んでいる。

In [14]:
x_seq = tf.convert_to_tensor([ [1.0]*5, [2.0]*5, [3.0]*5 ], dtype=tf.float32)
print("入力：\n", x_seq)

入力：
 tf.Tensor(
[[1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2.]
 [3. 3. 3. 3. 3.]], shape=(3, 5), dtype=float32)


作成した時系列データを一つ一つ処理をしていく。

In [21]:
output = rnn_layer(tf.reshape(x_seq, shape=(1, 3, 5)))

out_man = []

for t in range(len(x_seq)):

    xt = tf.reshape(x_seq[t], (1,5))
    print('Time step {} ->'.format(t))
    print('\t', x_seq[t])
    print('\t Input \t ', xt.numpy())
    
    ht = tf.matmul(xt, w_xh) + b_h
    print("\t Hidden \t : ", ht.numpy())
    
    if t > 0 :
        prev_o = out_man[t-1]
    else :
        prev_o = tf.zeros(shape=(ht.shape))
        
    ot = ht + tf.matmul(prev_o, w_oo)
    ot = tf.math.tanh(ot)
    
    out_man.append(ot)
    
    print('\t output (manual) : ', ot.numpy())
    print('\t SimpleRNN output : ', t, output[0][t].numpy(), "\n")

Time step 0 ->
	 tf.Tensor([1. 1. 1. 1. 1.], shape=(5,), dtype=float32)
	 Input 	  [[1. 1. 1. 1. 1.]]
	 Hidden 	 :  [[0.41464037 0.96012145]]
	 output (manual) :  [[0.39240566 0.74433106]]
	 SimpleRNN output :  0 [0.39240566 0.74433106] 

Time step 1 ->
	 tf.Tensor([2. 2. 2. 2. 2.], shape=(5,), dtype=float32)
	 Input 	  [[2. 2. 2. 2. 2.]]
	 Hidden 	 :  [[0.82928073 1.9202429 ]]
	 output (manual) :  [[0.80116504 0.9912947 ]]
	 SimpleRNN output :  1 [0.80116504 0.9912947 ] 

Time step 2 ->
	 tf.Tensor([3. 3. 3. 3. 3.], shape=(5,), dtype=float32)
	 Input 	  [[3. 3. 3. 3. 3.]]
	 Hidden 	 :  [[1.243921  2.8803642]]
	 output (manual) :  [[0.95468265 0.9993069 ]]
	 SimpleRNN output :  2 [0.95468265 0.9993069 ] 



## 長期的な相互作用の学習

時間方向に深いので、勾配消失もしくは勾配発散の問題が発生する。

- 勾配刈り込み
- T-BPTT
- LSTM


## LSTM

勾配消失問題を回s欠する方法として提唱された。
LSTMの構成要素はメモリセルで、標準的なRNNの隠れ層をこのメモリセルで置き換える。
文献を読むときには、LSTMには様々な種類が存在することに留意しておくこと。
共通するのは

- 忘却ゲート
- 入力ゲート
- 出力ゲート

の3種類のゲートを用いて、記憶を調整するということである。忘却ゲートと入力ゲートでは、その時刻$t$の情報を忘れるか覚えるかを判断し、出力ゲートではこれまでと同様隠れ状態ベクトルを出力する。
長期の情報を保持するセルの情報は $C$ として表され、$h$ とはまた別に導入される状態ベクトルであることに留意する。

### 忘却ゲート ( $f_t$ )

メモリセルを無限に成長させるのではなく、ゲートを通過させる情報と通過させない情報を決定する。

$$
f_t = \sigma \left(\boldsymbol{W}_{xf}\boldsymbol{x}^{(t)} + \boldsymbol{W}_{hf}\boldsymbol{h}^{(t-1)} + \boldsymbol{b}_f \right)
$$
 
- $\sigma$ : シグモイド関数 (活性化関数)
- $\boldsymbol{W}_{xf}$ : 時刻$t$の入力層から忘却ゲートへ結合するときの重み
- $\boldsymbol{W}_{hf}$ : ひとつ前の時刻 $t-1$ から忘却ゲートへ結合するときの重み


### 入力ゲート ( $i_t$ ) と候補値 ( $\tilde{C}_t$ )

$$
\boldsymbol{i}_t = \sigma \left(\boldsymbol{W}_{xi}\boldsymbol{x}^{(t)} + \boldsymbol{W}_{hi}\boldsymbol{h}^{(t-1)} + \boldsymbol{b}_i  \right)
$$

$$
\boldsymbol{\tilde{C}}_t = \mathrm{tanh} \left(\boldsymbol{W}_{xc}\boldsymbol{x}^{(t)} + \boldsymbol{W}_{hc}\boldsymbol{h}^{(t-1)} + \boldsymbol{b}_c  \right)
$$

時刻 $t$ のセル状態は次のように計算される

$$
C^{(t)} = \left( C^{(t-1)} \odot f_t \right) \oplus (i_t \odot \tilde{C}_t)
$$

日本語で書くと、時刻$t$のセル状態は

- ひとつ前の時刻 $t-1$ のセル状態 $\times$  忘却ゲート (過去のことを記憶するかどうかを判定）
- 時刻$t$での入力ゲート $\times$ 候補値 （現在のことを記憶するかどうかを判定）

の重ね合わせで、時刻$t$のセル状態($\simeq$ 記憶) を更新していく。

### 出力ゲート

$$
o_t = \sigma \left(\boldsymbol{W}_{xo}\boldsymbol{x}^{(t)} + \boldsymbol{W}_{ho}\boldsymbol{h}^{(t-1)} + \boldsymbol{b}_o  \right)
$$

これを用いて、現在時刻の隠れユニットは次のように計算される

$$
h^{(t)} = o_t \odot \mathrm{tanh} \left( C^{(t)} \right)
$$

# 実装

## §16.3.2 

テキスト文書を入力して、新しいテキストを生成できるモデルを開発する。入力情報は文字単位に分割され、RNNに一文字ずつ供給される。
RNNは受け取った一文字の次の文字を予測するように学習していく。例えば：

- データとして Hello world! を与える
- 'H', 'e', 'l', 'l', 'o', 'w', 'o', 'r', 'l', 'd', '!' に分割する
- 学習としては「H」を受け取ったら「e」、「e」を受け取ったら「l」...という様に次々に学習していく。


In [4]:
import numpy as np

with open("1268-0.txt", "r") as f:
    text = f.read()
    
start_index = text.find("THE MYSTERIOUS ISLAND")
end_index = text.find("End of the Project Gutenberg")
text = text[start_index:end_index]

char_set = set(text)

print("総文字数", len(text))
print("一意な文字種", len(char_set))

総文字数 1112350
一意な文字種 80


一般的なRNNライブラリでは文字列フォーマットの入力データには対処できない。
そのため、このテキストを数値に変換する（マッピングする）必要がある。簡単な辞書を作成し、読み取った文字列をそれに基づいて数値化していく。
ここではエンコーディング（文字列を数値化したことをここではエンコードと読んでいる）した結果を `text_encoded` に格納している。

In [10]:
chars_sorted = sorted(char_set)
char2int = {ch:idx for idx, ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)
text_encoded = np.array([char2int[ch] for ch in text], dtype=np.int32)

作成したエンコード済み文字列から Tensorflowの Datasetを作成する。

In [12]:
import tensorflow as tf
ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for ex in ds_text_encoded.take(5):
    print(ex.numpy(), char_array[ex.numpy()])

44 T
32 H
29 E
1  
37 M


In [17]:
seq_length = 40
chunk_size = seq_length + 1
ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

In [22]:
for example in ds_sequences.take(2):
    print('Input  (x)', repr(''.join(char_array[example[0].numpy()])))
    print('Target (y)', repr(''.join(char_array[example[1].numpy()])))

Input  (x) 'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'
Target (y) 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'
Input  (x) ' Anthony Matonak, and Trevor Carlson\n\n\n\n'
Target (y) 'Anthony Matonak, and Trevor Carlson\n\n\n\n\n'


In [23]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

### ここで keras について

- Embedding : 
    - 最初の入力層としてのみ使用できる
    - input_dim, output_dim 
- LSTM : 長短記憶
    - 
- Dense : 全結合ニューラルネット層
    - units (第一引数) : 出力の次元。一般的には `(batch_size, input_dim)` の入力を取り、 `(batch_size, units)` の次元を持つ出力を返す。
    - activation = None : 指定しなければ活性化なし

In [27]:
def build_model(vocab_size, embedding_dim, rnn_units):
    
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    
    return model

charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(1)

model = build_model(
    vocab_size=charset_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 256)         20480     
_________________________________________________________________
lstm (LSTM)                  (None, None, 512)         1574912   
_________________________________________________________________
dense (Dense)                (None, None, 80)          41040     
Total params: 1,636,432
Trainable params: 1,636,432
Non-trainable params: 0
_________________________________________________________________
